# Carregamento dos Dados

In [1]:
# Importar as bibliotecas necessárias
import pandas as pd

# URL do dataset Wine
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data'

# Nome das colunas em português BR
column_names_pt_br = [
    'classe',
    'alcool',
    'acido_malico',
    'cinzas',
    'alcalinidade_de_cinzas',
    'magnesio',
    'fenois_totais',
    'flavanoides',
    'fenois_nao_flavanoides',
    'proantocianinas',
    'intensidade_de_cor',
    'matiz',
    'od280_od315_de_vinhos_diluidos',
    'prolina'
]

# Ler o arquivo CSV com as colunas especificadas, definindo a coluna 'classe' como object
vinhos = pd.read_csv(url, names=column_names_pt_br, dtype={'classe': object})

# Visualizar as primeiras linhas do dataset
vinhos.head()


,classe,alcool,acido_malico,cinzas,alcalinidade_de_cinzas,magnesio,fenois_totais,flavanoides,fenois_nao_flavanoides,proantocianinas,intensidade_de_cor,matiz,od280_od315_de_vinhos_diluidos,prolina
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


## Pré-Processamento dos Dados

In [2]:
# Separamos o dataset em variáveis de entrada (X) e a variável-alvo (y).

# Separar as características (X) e os rótulos (y)
X = vinhos.drop(columns=['classe'])
y = vinhos['classe']

# Verificar as dimensões dos dados
print(f"Dimensão de X: {X.shape}")
print(f"Dimensão de y: {y.shape}")


Dimensão de X: (178, 13)
Dimensão de y: (178,)


## Configuração do K-Fold Cross-Validation

In [3]:
# Usaremos KFold para dividir o dataset em 10 folds, garantindo replicabilidade com random_state.

from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import numpy as np

# Função para realizar o k-fold cross-validation e calcular métricas
def cross_validation_knn(X, y, k, random_state):
    kf = KFold(n_splits=10, shuffle=True, random_state=random_state)
    acuracias = []

    # Loop pelos folds
    for train_index, test_index in kf.split(X):
        # Dividir os dados em treino e teste
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        # Treinar o classificador KNN
        knn = KNeighborsClassifier(n_neighbors=k)
        knn.fit(X_train, y_train)

        # Fazer previsões e calcular a acurácia
        y_pred = knn.predict(X_test)
        acuracias.append(accuracy_score(y_test, y_pred))

    # Retornar média e desvio padrão das acurácias
    return np.mean(acuracias), np.std(acuracias)


## Realização dos Experimentos

In [4]:
# Executaremos o KNN para k=3 e k=5, com os valores de random_state 42, 17 e 24.

# Configurações do experimento
random_states = [42, 17, 24]
k_values = [3, 5]

# Armazenar resultados
resultados = []

# Loop pelos random_states e valores de k
for random_state in random_states:
    for k in k_values:
        media, desvio = cross_validation_knn(X, y, k, random_state)
        resultados.append({
            'Random State': random_state,
            'k': k,
            'Média': media,
            'Desvio Padrão': desvio
        })

# Criar DataFrame com os resultados
df_resultados = pd.DataFrame(resultados)
df_resultados


,Random State,k,Média,Desvio Padrão
0,42,3,0.703595,0.139187
1,42,5,0.664052,0.100075
2,17,3,0.698693,0.104030
3,17,5,0.687582,0.112118
4,24,3,0.712745,0.096816
5,24,5,0.712745,0.093973


## Apresentação e Análise dos Resultados

In [5]:
# Mostrar resultados ordenados por random_state e k
df_resultados = df_resultados.sort_values(by=['Random State', 'k'])
print(df_resultados)

# Identificar o melhor resultado
melhor_modelo = df_resultados.loc[df_resultados['Média'].idxmax()]
print("\nMelhor configuração:")
print(melhor_modelo)

#    O DataFrame df_resultados exibe a acurácia média e o desvio padrão para cada combinação de random_state e k.
#    O melhor modelo é identificado com base na maior acurácia média.

   Random State  k     Média  Desvio Padrão
2            17  3  0.698693       0.104030
3            17  5  0.687582       0.112118
4            24  3  0.712745       0.096816
5            24  5  0.712745       0.093973
0            42  3  0.703595       0.139187
1            42  5  0.664052       0.100075

Melhor configuração:
Random State     24.000000
k                 3.000000
Média             0.712745
Desvio Padrão     0.096816
Name: 4, dtype: float64
